In [1]:
!pip install Flask requests

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# from flask import Flask, jsonify, request
# import threading

# app = Flask(__name__)

# @app.route('/hello', methods=['GET'])
# def hello_world():
#     return jsonify(message="you called?")

# @app.route('/echo', methods=['POST'])
# def echo():
#     data = request.json
#     return jsonify(data)

# def run_server():
#     app.run(port=5000, debug=False)

# # Run the server in a thread to avoid blocking the notebook
# thread = threading.Thread(target=run_server)
# thread.start()

In [4]:
# import requests

# # Sending a GET request to the /hello endpoint
# response = requests.get('http://127.0.0.1:5000/hello')
# print(response.json())

# # Sending a POST request to the /echo endpoint
# data = {'message': 'you are the server!'}
# response = requests.post('http://127.0.0.1:5000/echo', json=data)
# print(response.json())


127.0.0.1 - - [24/Jul/2024 02:33:48] "GET /hello HTTP/1.1" 200 -
127.0.0.1 - - [24/Jul/2024 02:33:48] "POST /echo HTTP/1.1" 200 -


{'message': 'you called?'}
{'message': 'you are the server!'}


In [1]:
from flask import Flask, send_file, request
import requests as req
import threading
import os

app = Flask(__name__)

VIDEO_URL = 'http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/BigBuckBunny.mp4'
VIDEO_PATH = 'downloaded_video.mp4'

@app.route('/get_video', methods=['GET'])
def get_video():
    # Download the video
    video_data = req.get(VIDEO_URL)
    
    # Save the video to a file
    with open(VIDEO_PATH, 'wb') as f:
        f.write(video_data.content)
    
    # Return the video file
    return send_file(VIDEO_PATH, mimetype='video/mp4')

def run_server():
    app.run(port=5000, debug=False)

# Run the server in a thread to avoid blocking the notebook
server_thread = threading.Thread(target=run_server)
server_thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [2]:
import requests
import cv2
import numpy as np
import time

# Wait a bit for the server to start
time.sleep(2)

# Request the video from the server
response = requests.get('http://127.0.0.1:5000/get_video')

# Save the video to a file
with open('client_video.mp4', 'wb') as f:
    f.write(response.content)

# Display the video using OpenCV
cap = cv2.VideoCapture('client_video.mp4')

if not cap.isOpened():
    print("Error: Could not open video.")
else:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imshow('Video', frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


127.0.0.1 - - [24/Jul/2024 09:36:34] "GET /get_video HTTP/1.1" 200 -


In [ ]:
# server.py

import asyncio
import json
import time
import argparse

# Configuration via CLI
parser = argparse.ArgumentParser(description="Volumetric Streaming Server")
parser.add_argument('--host', type=str, default='0.0.0.0')
parser.add_argument('--buf-port', type=int, default=9001)
parser.add_argument('--hb-port', type=int, default=9002)
parser.add_argument('--frame-port', type=int, default=9003)
parser.add_argument('--frame-rate', type=int, default=30)
parser.add_argument('--report-interval', type=float, default=1.0)
args = parser.parse_args()

clients = {}  # addr -> {'buf_writer', 'prediction'}

async def handle_buffer_commands(reader, writer):
    addr = writer.get_extra_info('peername')
    clients[addr] = {'buf_writer': writer, 'prediction': 0.0}
    print(f"[BufferCmd] Client {addr} connected")
    try:
        while True:
            await asyncio.sleep(1)
    except asyncio.CancelledError:
        writer.close()
        await writer.wait_closed()

async def handle_heartbeat(reader, writer):
    addr = writer.get_extra_info('peername')
    print(f"[Heartbeat] Client {addr} connected")
    try:
        while True:
            data = await reader.readline()
            if not data:
                break
            msg = json.loads(data)
            # echo back with timestamp
            writer.write((json.dumps({'timestamp': msg['timestamp']}) + '\n').encode())
            await writer.drain()
            # record client's prediction if provided
            if 'prediction' in msg:
                clients[addr]['prediction'] = msg['prediction']
    except asyncio.CancelledError:
        pass
    writer.close()
    await writer.wait_closed()
    clients.pop(addr, None)

async def frame_stream():
    frame_interval = 1.0 / args.frame_rate
    frame_id = 0
    while True:
        for addr, info in clients.items():
            writer = info['buf_writer']
            cmd = {'cmd': 'frame', 'frame_id': frame_id}
            writer.write((json.dumps(cmd) + '\n').encode())
            await writer.drain()
        frame_id += 1
        await asyncio.sleep(frame_interval)

async def control_loop():
    while True:
        if clients:
            preds = [info['prediction'] for info in clients.values()]
            max_pred = max(preds) if preds else 0
            for addr, info in clients.items():
                buf_size = max_pred - info['prediction']
                cmd = {'buffer_size': buf_size}
                writer = info['buf_writer']
                writer.write((json.dumps(cmd) + '\n').encode())
                await writer.drain()
        await asyncio.sleep(args.report_interval)

async def main():
    server_buf = await asyncio.start_server(handle_buffer_commands, args.host, args.buf_port)
    server_hb  = await asyncio.start_server(handle_heartbeat, args.host, args.hb_port)
    print(f"Server listening on {args.host} ports BUF:{args.buf_port}, HB:{args.hb_port}")
    await asyncio.gather(
        server_buf.serve_forever(),
        server_hb.serve_forever(),
        frame_stream(),
        control_loop()
    )

if __name__ == '__main__':
    asyncio.run(main())






In [ ]:
# client.py

import asyncio
import json
import time
import csv
import argparse

parser = argparse.ArgumentParser(description="Volumetric Streaming Client")
parser.add_argument('--server-host', type=str, default='127.0.0.1')
parser.add_argument('--buf-port', type=int, default=9001)
parser.add_argument('--hb-port', type=int, default=9002)
parser.add_argument('--frame-port', type=int, default=9003)
parser.add_argument('--client-id', type=int, required=True)
parser.add_argument('--history-k', type=int, default=10)
parser.add_argument('--heartbeat-interval', type=float, default=1.0)
args = parser.parse_args()

# Logging setup
log_file = f"logs/client_{args.client_id}.csv"
csv_file = open(log_file, 'w', newline='')
writer = csv.writer(csv_file)
writer.writerow(['frame_id', 'render_ts', 'buffer_size', 'latency', 'prediction', 'residual'])

latencies = []
predictions = []
residuals = []
buffer_size = 0.0
K = args.history_k

def ewma_predict(history):
    alpha = 0.3
    if len(history) <= 1:
        return history[-1] if history else 0.0
    pred = history[-1]
    for h in reversed(history[:-1]):
        pred = alpha * pred + (1 - alpha) * h
    return pred

async def heartbeat_loop(reader, writer_hb):
    global latencies, predictions, residuals
    while True:
        send_ts = time.time()
        msg = {'timestamp': send_ts, 'prediction': predictions[-1] if predictions else 0.0}
        writer_hb.write((json.dumps(msg) + '\n').encode())
        await writer_hb.drain()
        data = await reader.readline()
        if not data:
            break
        resp = json.loads(data)
        recv_ts = time.time()
        rtt = recv_ts - resp['timestamp']
        one_way = rtt / 2
        latencies.append(one_way)
        if len(latencies) > K:
            latencies.pop(0)
        pred = ewma_predict(latencies)
        predictions.append(pred)
        res = abs(pred - one_way)
        residuals.append(res)
        await asyncio.sleep(args.heartbeat_interval)

async def buffer_listener(reader, writer_buf):
    global buffer_size
    while True:
        data = await reader.readline()
        if not data:
            break
        cmd = json.loads(data)
        if 'buffer_size' in cmd:
            buffer_size = cmd['buffer_size']

async def frame_listener(reader, writer_frame):
    global buffer_size
    while True:
        line = await reader.readline()
        if not line:
            break
        frame = json.loads(line)
        if frame.get('cmd') == 'frame':
            frame_id = frame['frame_id']
            # simulate render
            time.sleep(0.001)
            render_ts = time.time()
            latency = latencies[-1] if latencies else 0.0
            prediction = predictions[-1] if predictions else 0.0
            residual = residuals[-1] if residuals else 0.0
            writer.writerow([frame_id, render_ts, buffer_size, latency, prediction, residual])
            csv_file.flush()

async def main():
    rb, wb = await asyncio.open_connection(args.server_host, args.buf_port)
    rh, wh = await asyncio.open_connection(args.server_host, args.hb_port)
    rf, wf = await asyncio.open_connection(args.server_host, args.frame_port)

    await asyncio.gather(
        heartbeat_loop(rh, wh),
        buffer_listener(rb, wb),
        frame_listener(rf, wf)
    )

if __name__ == '__main__':
    asyncio.run(main())